In [1]:
import sys
sys.path.insert(0, '/projekti/wikidata')
import os
from collections import defaultdict, Counter
from text_utils import remove_reg, tokenise, get_provenance
from itertools import combinations, product
import json
import numpy as np
from tqdm.auto import tqdm
from wiki_labels import qid_lab_get, qid_lab
import gzip
import pickle

#print(qid_lab_get(177053, include_alt=True)['mr'])
#print(qid_lab_get(191701, include_alt=True)['mba'])

In [19]:
# dic = pickle.load(open('/projekti/mondodb/lists/wiki_freq_dict.pickle', 'rb'))
dic['hr_Latn_dae']

Counter()

In [14]:
named_ent

{'wiki_id': 'per'}

In [ ]:
fo = gzip.open('/projekti/mondodb/lists/wikiname.tsv.gz', 'wt')
for l in tqdm(gzip.open('/backup/wikidata/wikiname.jsonl.gz', 'rt'), total=854769):
    j = json.loads(l)
    wid = int(j['wiki_id'][1:])
    labs = qid_lab_get(wid, include_alt=True, return_alt=True)
    for lab, langs in labs.items():
        for lang in langs:
            lang = lang[0]
            if '_' in lang:
                lang = lang.split('_')[0]
            elif '-' in lang:
                lang = lang.split('-')[0]
            prov = get_provenance(lab, lang, no_countries=True)
            t = j['type'].replace('N.', '').lower()
            if prov:
                fo.write(f'{t}\t{lab}\t{prov[0]}\n')

    #w1, w2 = qid_lab_get(w1), qid_lab_get(w2)
fo.close()

In [ ]:
pairs = Counter()
for l in tqdm(gzip.open('/backup/wikidata/wikiname.jsonl', 'rt'), total=854769):
    j = json.loads(l)
    sim = [int(a.replace('WIKI_Q', '').replace('Q', '')) for a in j['variants']+[j['wiki_id']] if 'P' not in a]
    for w1, w2 in combinations(sim, 2):
        w1, w2 = qid_lab_get(w1), qid_lab_get(w2)
        for v1, v2 in product(w1.keys(), w2.keys()):
            if v1<v2 and w1[v1].intersection(w2[v2]):
                pairs[(v1, v2)] += 1
    #break
len(pairs)

In [ ]:
import oracledb

tls = 'db202201301230_high'
tls = 'db202201301230_high'
psw = open('/projekti/wikidata/orapsw.txt').read()
connection = oracledb.connect(user='mondo', password=psw, dsn=tls, encoding="UTF-8",
                                   config_dir='/usr/lib/oracle/21/client64/lib/network/admin/',
                              wallet_location='/usr/lib/oracle/21/client64/lib/network/admin/', wallet_password=psw)
cur = connection.cursor()
sql = 'create table wiki_samaas (w1 varchar2(100), w2 varchar2(100), freq number(7,0))'
#NAME2NAME BIBLICAL_TRANS_BIBLE BIBLICAL_NAME_TRANSLATION
t = cur.execute(sql)
for (n1, n2), v in pairs.items():
    cur.execute('insert into wiki_samaas values (:1, :2, :3)', (n1, n2, v))

connection.commit()

In [ ]:
cur.execute("alter table name2name add wiki_sameas number (5,0)")
cur.execute("""MERGE into name2name nn
using (select * from wiki_samaas) B
on (B.w1=nn.N1 and B.w2=nn.N2)
when MATCHED then update set wiki_sameas = B.freq
when not MATCHED then insert (N1, N2, wiki_sameas) values (b.w1, b.w2, b.freq)
""")
connection.commit()

In [ ]:
toks = defaultdict(Counter)
for tok, v in tqdm(names.items(), total=len(names)):
    for t_prov, cnt in v.items():
        src, t, lang, script, cc = t_prov.split('_')
        tok = tok.replace('_', ' ')
        if len(tok)>0:
            toks[f'{tok}_{lang}_{script}'][(src, t)] += cnt
            if tok[-1] == '.' and tok.count('.') == 1:
                toks[f'{tok[:-1]}_{lang}_{script}'][(src+'.', t)] += cnt
toks = dict(toks)
toks['dr_en_Latn'], toks['mba_en_Latn']

In [ ]:
t2set = defaultdict(set)
prov2set = defaultdict(set)

for t, c in t2i.items():
    t = t.split('_')
    t2set['_'.join(t[:3])].add(c)    
    prov2set['_'.join(t[1:3])].add('_'.join(t[1:]))
t2set = dict(t2set)    
i2t = {k: v for v, k in t2i.items()}
prov2set = dict(prov2set)    
i2t = {k: v for v, k in t2i.items()}
t2set['tit_hr_Latn'], prov2set['de_Latn']

In [ ]:
freqs = {}
for k, v in tqdm(toks.items(), total=len(toks)):
    (tok, provt) = k.split('_', 1) 
    for (src, tip), cnt in v.items():
        if (tok, provt) not in freqs:
            freqs[(tok, provt)] = {'fn0': 0, 'fn1': 0, 'fn2': 0, 'ln': 0, 'tit': 0, }
        freqs[(tok, provt)][tip] += cnt

freqs[('prof', 'hr_Latn')]

In [ ]:
freqs2 = {}
nema = Counter()
for k, v in tqdm(freqs.items(), total=len(freqs)):
    tok, provt = k
    tips = [k for k in v if v[k]>0]
    if provt in prov2set:
        for tip in tips:
            if tip == 'tit':
                allprovs = [provt+'_']
            else:
                allprovs = [a for a in prov2set[provt] if not a.endswith('_')]
            cnt = v[tip]
            cnt_all = sum(v.values())
            if cnt_all > 0: 
                for prov in allprovs:
                    if f'{tip}_{prov}' in t2i:
                        provi = t2i[f'{tip}_{prov}']
                        freqs2[(tok, provi)] = (cnt, round(cnt*100 / cnt_all,2))
                    else:
                        nema[f'{tip}_{prov}'] += 1
len(nema), sum(nema.values()), freqs2[('doctor', t2i['tit_en_Latn_'])], freqs2[('doctor', t2i['ln_en_Latn_US'])], freqs2[('prof', t2i['tit_hr_Latn_'])]

In [ ]:
# to Oracle
import cx_Oracle
tls = 'db202201301230_high'
connection = cx_Oracle.connect(user='mondo', password=open('orapsw.txt').read(),dsn=tls,encoding="UTF-8")
cur = connection.cursor()


In [ ]:
try:
    cur.execute('drop table mondo.wiki_name_parts purge') 
except:
    pass
cur.execute("""create table mondo.wiki_name_parts (token varchar2(200), lang varchar2(10), fn0 number(6, 0), fn1 number(6, 0), fn2 number(6, 0),
            ln number(6, 0), tit number(6, 0), CONSTRAINT pk_locations PRIMARY KEY (token, lang)) ORGANIZATION INDEX""")
print('loading ...')
batch = []
br = 0
for (tok, lng), cnt in tqdm(freqs.items(), total=len(freqs)):
    if len(bytes(tok, 'utf-8')) < 190:
        batch.append((tok, lng, cnt['fn0'], cnt['fn1'], cnt['fn2'], cnt['ln'], cnt['tit'] ))
    if len(batch)>10_048:
        cur.executemany('insert into mondo.wiki_name_parts values (:1, :2, :3, :4, :5, :6, :7) ', batch) 
        connection.commit()
        batch = []
    br += 1
cur.executemany('insert into mondo.wiki_name_parts values (:1, :2, :3, :4, :5, :6, :7) ', batch) 
connection.commit()
print(br)

In [ ]:
cur.execute('select count(*) from mondo.wiki_name_parts').fetchall()

In [ ]:
# select m.n, (select t.t from  mondo.TYPEID t where t.id=m.t) t, c from MONDO.NAME_M m where n='dr' and
# not exists (select 1 from mondo.TYPEID where id=m.t and regexp_like(t, 'title_.+') )  and
# exists (select 1 from mondo.wiki_name_parts w where w.token=m.n) and
# not exists (select 1 from mondo.wiki_name_parts w where w.token=m.n and w.PROV=m.T and w.RATIO>5);

# AD HOC cleaning 

cur.execute("""
delete from MONDO.NAME_M m where
not exists (select 1 from mondo.TYPEID where id=m.t and regexp_like(t, 'title_.+') )  and
exists (select 1 from mondo.wiki_name_parts w where w.token=m.n) and
not exists (select 1 from mondo.wiki_name_parts w where w.token=m.n and w.PROV=m.T and w.RATIO>5)
""")
connection.commit()

In [ ]:
cur.execute("select m.n, t.t from mondo.name_m m, mondo.TYPEID t where t.id=m.t and m.n='mr.'").fetchall()